In [66]:
import sys
import os
import pandas as pd
# Add your project folder path to sys.path
sys.path.append(os.path.abspath("F:/Customer-Retention-Prediction"))
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV ,StratifiedKFold
from utils.utils import save_model_or_data,load_data
from sklearn.metrics import accuracy_score


In [67]:
rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50,100, 200],
    'max_depth': [8, 16, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 0.5, 0.8]
}

In [68]:
X_train=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_train_scaled.csv")
y_train=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_train.csv")
X_val=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_val_scaled.csv")
y_val=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_val.csv")  

#### Feature selelction

In [69]:
# Train initial model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Get feature importances
importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf.feature_importances_
}).sort_values(by='importance', ascending=False)

print(importances)

f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


                       feature  importance
6   Average_Transaction_Amount    0.125488
10      Last_Purchase_Days_Ago    0.122268
3                  Total_Spend    0.114109
2                Annual_Income    0.111641
5             Num_of_Purchases    0.109984
0                          Age    0.101935
4            Years_as_Customer    0.081198
7               Num_of_Returns    0.064949
8      Num_of_Support_Contacts    0.045296
9           Satisfaction_Score    0.041592
1                       Gender    0.035330
12          Promotion_Response    0.027838
11                Email_Opt_In    0.018374


In [70]:
# Keep top N features (example: top 5)
top_features = importances['feature'].iloc[:4]
X_train_top = X_train[top_features]
X_val_top = X_val[top_features]

In [71]:
# Retrain RF on top features
rf_top = RandomForestClassifier(random_state=42)
rf_top.fit(X_train_top, y_train)


f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [72]:
# Evaluate
val_acc = rf_top.score(X_val_top, y_val)
print("Validation accuracy (RF top features):", val_acc)

Validation accuracy (RF top features): 0.475


In [73]:
X_train.head()

,Age,Gender,Annual_Income,Total_Spend,Years_as_Customer,Num_of_Purchases,Average_Transaction_Amount,Num_of_Returns,Num_of_Support_Contacts,Satisfaction_Score,Last_Purchase_Days_Ago,Email_Opt_In,Promotion_Response
0,0.000000,0.5,0.613406,0.953714,0.611111,0.530612,0.868653,0.444444,1.00,0.25,0.409972,0.0,1.0
1,0.745098,0.0,0.922905,0.830412,0.500000,0.908163,0.169818,0.888889,0.50,0.75,0.637119,1.0,0.0
2,0.705882,0.0,0.271203,0.775999,1.000000,0.132653,0.807317,0.222222,0.25,0.00,0.922438,1.0,1.0
3,0.843137,0.5,0.554649,0.901427,0.777778,0.132653,0.348987,1.000000,0.00,0.00,0.022161,0.0,0.5
4,0.647059,0.0,0.683513,0.881428,0.388889,0.795918,0.780494,0.555556,0.75,0.75,0.210526,1.0,1.0


In [74]:
y_train.head()

,Target_Churn
0,0
1,0
2,0
3,1
4,0


In [75]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(rf_top, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=2)
grid.fit(X_train_top, y_train)

print("Best Parameters:", grid.best_params_)
print("Best CV Score:", grid.best_score_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Parameters: {'max_depth': 16, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Best CV Score: 0.571875


In [76]:
best_rf = grid.best_estimator_

# Evaluate on validation set
y_val_pred = best_rf.predict(X_val_top)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))

Validation Accuracy: 0.4625


##### Now find acc on train set and compare with validation for checking overfitting and underfitting

In [77]:
y_train_pred = best_rf.predict(X_train_top)
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))

Training Accuracy: 0.9828125


In [78]:
#### That means overfitting is there as training accuracy is much higher than validation accuracy. So we can tune 
# hyperparameters more to reduce overfitting.